# Full audio

## Feature

### Set up

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL FULL AUDIO')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [3]:
n_mfcc = 0

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet'

coswara_feature = os.path.join(root, 'full_audio/melspectrogram_coswara_features.npy')

coughvid_feature = os.path.join(root, 'full_audio/melspectrogram_coughvid_features.npy')

meta_coswara = os.path.join(root, 'full_audio/coswara_metadata_for_label_matching.csv')
meta_coughvid = os.path.join(root, 'full_audio/coughvid_metadata_for_label_matching.csv')

### Feature preparation

#### Metadata

In [4]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,...,stridor_3,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,cough_path,label,error
0,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,healthy,0
1,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,healthy,0
2,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/0012c608-33d0-4ef7-bde3-75a0b1a0024e.webm,NaN,0
3,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f.webm,healthy,0
4,001c85a8-cc4d-4921-9297-848be52d4715,2020-04-17T15:24:35.822355+00:00,0.0735,40.6,-3.6,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data/coughvid/coughvid_cough_data/001c85a8-cc4d-4921-9297-848be52d4715.webm,NaN,0


In [6]:
logger.info(meta_df.shape)

2022-05-12 15:28:37,118 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - (20072, 43)


In [7]:
# mel_feature = np.load(coughvid_feature)
meta_df = meta_df[meta_df['error'] == 0].reset_index(drop=True)

In [ ]:
mel_feature[0].shape, len(mel_feature), meta_df.shape

((224, 246), 20072, (20072, 43))

In [5]:
DROP_COLS = [
             'uuid',
             'error',
             'datetime',
             'cough_path',
             'cough_detected',
             ]

meta_df.isna().sum() / 20072 *100 > 50

uuid                     False
datetime                 False
cough_detected           False
latitude                 False
longitude                False
age                      False
gender                   False
respiratory_condition    False
fever_muscle_pain        False
status                   False
quality_1                 True
cough_type_1              True
dyspnea_1                 True
wheezing_1                True
stridor_1                 True
choking_1                 True
congestion_1              True
nothing_1                 True
diagnosis_1               True
severity_1                True
quality_2                 True
cough_type_2              True
dyspnea_2                 True
wheezing_2                True
stridor_2                 True
choking_2                 True
congestion_2              True
nothing_2                 True
diagnosis_2               True
severity_2                True
quality_3                 True
dyspnea_3                 True
wheezing

In [ ]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
status                    object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3 

In [ ]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
status                   0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
c

In [8]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [9]:
meta_df.shape

(20072, 38)

In [ ]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-04-30 09:14:47,521 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - {0.0: 8562, 1.0: 1742}


In [ ]:
meta_df.label.value_counts().sum()

10304

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

##### 2d

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-04-30 09:14:48,388 - COUGHVID - HYBRID MODEL FULL AUDIO - INFO - 1742, 8562


In [ ]:
# get features for each class
feature_0 = mel_feature[index_0]
feature_1 = mel_feature[index_1]

In [ ]:
print(len(feature_1))

1742


In [ ]:
del mel_feature

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:6850], feature_1[:1394]), axis=0)
  val = np.concatenate((feature_0[6850:7706], feature_1[1394:1568]), axis=0)
  test = np.concatenate((feature_0[7706:], feature_1[1568:]), axis=0)

  y_train = pd.concat([label0[:6850], label1[:1394]], ignore_index=True)
  y_val = pd.concat([label0[6850:7706], label1[1394:1568]], ignore_index=True)
  y_test = pd.concat([label0[7706:], label1[1568:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(8244, 224, 246)


In [ ]:
X_train_2d.shape, X_val_2d.shape, X_test_2d.shape

((8244, 224, 246), (1030, 224, 246), (1030, 224, 246))

In [ ]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 224, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
X_train_1d.shape

(8244, 37, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(merged_model, to_file='model_final.png')
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
from datetime import datetime
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'full_audio_coughvid_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 79s 67ms/step - loss: 0.5016 - auc: 0.8345 - val_loss: 0.5801 - val_auc: 0.8334
Epoch 2/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4653 - auc: 0.8382 - val_loss: 0.5399 - val_auc: 0.8226
Epoch 3/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4593 - auc: 0.8435 - val_loss: 0.4695 - val_auc: 0.8346
Epoch 4/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4570 - auc: 0.8425 - val_loss: 0.4555 - val_auc: 0.8362
Epoch 5/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4556 - auc: 0.8430 - val_loss: 0.4574 - val_auc: 0.8286
Epoch 6/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4564 - auc: 0.8408 - val_loss: 0.4587 - val_auc: 0.8285
Epoch 7/40
1031/1031 [==============================] - 65s 63ms/step - loss: 0.4547 - auc: 0.8421 - val_loss: 0.4580 - val_auc: 0.8301
Epoch 8/40
1031/1031 [==========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 69s 119ms/step - loss: 0.5808 - auc: 0.8406 - val_loss: 0.4921 - val_auc: 0.8312
Epoch 2/40
516/516 [==============================] - 61s 117ms/step - loss: 0.4623 - auc: 0.8457 - val_loss: 0.4879 - val_auc: 0.8329
Epoch 3/40
516/516 [==============================] - 60s 117ms/step - loss: 0.4585 - auc: 0.8414 - val_loss: 0.4584 - val_auc: 0.8391
Epoch 4/40
516/516 [==============================] - 60s 117ms/step - loss: 0.4595 - auc: 0.8397 - val_loss: 0.4740 - val_auc: 0.8381
Epoch 5/40
516/516 [==============================] - 60s 117ms/step - loss: 0.4566 - auc: 0.8445 - val_loss: 0.4620 - val_auc: 0.8283
Epoch 6/40
516/516 [==============================] - 60s 117ms/step - loss: 0.4554 - auc: 0.8460 - val_loss: 0.4615 - val_auc: 0.8368
Epoch 7/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4528 - auc: 0.8488 - val_loss: 0.4572 - val_auc: 0.8391
Epoch 8/40
33/33 [==============================] - 1s 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 65s 859ms/step - loss: 0.7945 - auc: 0.8281 - val_loss: 0.5476 - val_auc: 0.8330
Epoch 2/40
65/65 [==============================] - 54s 836ms/step - loss: 0.4598 - auc: 0.8429 - val_loss: 0.5398 - val_auc: 0.8362
Epoch 3/40
65/65 [==============================] - 55s 841ms/step - loss: 0.4536 - auc: 0.8507 - val_loss: 0.5100 - val_auc: 0.8393
Epoch 4/40
65/65 [==============================] - 54s 839ms/step - loss: 0.4533 - auc: 0.8517 - val_loss: 0.4946 - val_auc: 0.8426
Epoch 5/40
65/65 [==============================] - 55s 842ms/step - loss: 0.4523 - auc: 0.8517 - val_loss: 0.4802 - val_auc: 0.8409
Epoch 6/40
65/65 [==============================] - 55s 845ms/step - loss: 0.4482 - auc: 0.8554 - val_loss: 0.4696 - val_auc: 0.8409
Epoch 7/40
65/65 [==============================] - 55s 840ms/step - loss: 0.4460 - auc: 0.8585 - val_loss: 0.4699 - val_auc: 0.8408
Epoch 8/40
65/65 [==============================] - 55s 841ms/step - 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/full_audio_coughvid_mel_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(8244, 224, 246)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(8244, 224, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(8244, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([8244, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = os.path.join(SAVED_MODEL_PATH, '/coswara_melme_hybrid_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_full_audio_coughvid_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1031/1031 [==============================] - 78s 67ms/step - loss: 0.5015 - auc: 0.8329 - val_loss: 0.6306 - val_auc: 0.8309
Epoch 2/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4633 - auc: 0.8435 - val_loss: 0.5791 - val_auc: 0.8361
Epoch 3/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4602 - auc: 0.8431 - val_loss: 0.4817 - val_auc: 0.8383
Epoch 4/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4573 - auc: 0.8421 - val_loss: 0.4753 - val_auc: 0.8274
Epoch 5/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4566 - auc: 0.8420 - val_loss: 0.4563 - val_auc: 0.8328
Epoch 6/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4552 - auc: 0.8418 - val_loss: 0.4570 - val_auc: 0.8242
Epoch 7/40
1031/1031 [==============================] - 66s 64ms/step - loss: 0.4548 - auc: 0.8431 - val_loss: 0.4565 - val_auc: 0.8318
Epoch 8/40
1031/1031 [==========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_8_0.001/assets


Epoch 1/40
516/516 [==============================] - 68s 118ms/step - loss: 0.5797 - auc: 0.8314 - val_loss: 0.4857 - val_auc: 0.8296
Epoch 2/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4583 - auc: 0.8429 - val_loss: 0.4621 - val_auc: 0.8325
Epoch 3/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4568 - auc: 0.8435 - val_loss: 0.4577 - val_auc: 0.8396
Epoch 4/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4546 - auc: 0.8458 - val_loss: 0.4635 - val_auc: 0.8311
Epoch 5/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4543 - auc: 0.8456 - val_loss: 0.4713 - val_auc: 0.8284
Epoch 6/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4532 - auc: 0.8480 - val_loss: 0.4743 - val_auc: 0.8292
Epoch 7/40
516/516 [==============================] - 60s 116ms/step - loss: 0.4519 - auc: 0.8487 - val_loss: 0.4570 - val_auc: 0.8403
Epoch 8/40
516/516 [==============================] - 6

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_64_0.001/assets


Epoch 1/40
65/65 [==============================] - 65s 858ms/step - loss: 0.7874 - auc: 0.8305 - val_loss: 0.5958 - val_auc: 0.8371
Epoch 2/40
65/65 [==============================] - 55s 838ms/step - loss: 0.4583 - auc: 0.8450 - val_loss: 0.5583 - val_auc: 0.8448
Epoch 3/40
65/65 [==============================] - 54s 836ms/step - loss: 0.4534 - auc: 0.8489 - val_loss: 0.5662 - val_auc: 0.8380
Epoch 4/40
65/65 [==============================] - 54s 835ms/step - loss: 0.4504 - auc: 0.8530 - val_loss: 0.5717 - val_auc: 0.8344
Epoch 5/40
65/65 [==============================] - 55s 839ms/step - loss: 0.4500 - auc: 0.8552 - val_loss: 0.5487 - val_auc: 0.8309
Epoch 6/40
65/65 [==============================] - 55s 851ms/step - loss: 0.4456 - auc: 0.8614 - val_loss: 0.5221 - val_auc: 0.8316
Epoch 7/40
33/33 [==============================] - 1s 38ms/step - loss: 0.4800 - auc: 0.8416


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/std_full_audio_coughvid_mel_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(8244, 224, 246)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(13700, 224, 246, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(13700, 37, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([13700, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './coswara_hybrid_mel_bestweight.h5')

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_full_audio_coughvid_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
1713/1713 [==============================] - 116s 63ms/step - loss: 0.5674 - auc: 0.7811 - val_loss: 0.7207 - val_auc: 0.8290
Epoch 2/40
1713/1713 [==============================] - 107s 62ms/step - loss: 0.4808 - auc: 0.8451 - val_loss: 0.5115 - val_auc: 0.8412
Epoch 3/40
1713/1713 [==============================] - 106s 62ms/step - loss: 0.4640 - auc: 0.8563 - val_loss: 1.1094 - val_auc: 0.4534
Epoch 4/40
1713/1713 [==============================] - 106s 62ms/step - loss: 0.4529 - auc: 0.8618 - val_loss: 0.5434 - val_auc: 0.8133
Epoch 5/40
1713/1713 [==============================] - 106s 62ms/step - loss: 0.4448 - auc: 0.8687 - val_loss: 5.6402 - val_auc: 0.1672
Epoch 6/40
1713/1713 [==============================] - 106s 62ms/step - loss: 0.4202 - auc: 0.8824 - val_loss: 0.9756 - val_auc: 0.4989
Epoch 7/40
33/33 [==============================] - 2s 38ms/step - loss: 0.6113 - auc: 0.7395


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_8_0.001/assets


Epoch 1/40
857/857 [==============================] - 105s 115ms/step - loss: 0.6505 - auc: 0.7356 - val_loss: 0.8968 - val_auc: 0.5691
Epoch 2/40
857/857 [==============================] - 97s 114ms/step - loss: 0.4714 - auc: 0.8511 - val_loss: 0.6207 - val_auc: 0.7369
Epoch 3/40
857/857 [==============================] - 98s 114ms/step - loss: 0.4383 - auc: 0.8720 - val_loss: 0.5424 - val_auc: 0.8328
Epoch 4/40
857/857 [==============================] - 98s 114ms/step - loss: 0.4306 - auc: 0.8762 - val_loss: 0.6205 - val_auc: 0.7622
Epoch 5/40
857/857 [==============================] - 98s 114ms/step - loss: 0.4201 - auc: 0.8833 - val_loss: 0.8981 - val_auc: 0.5740
Epoch 6/40
857/857 [==============================] - 97s 114ms/step - loss: 0.4066 - auc: 0.8906 - val_loss: 1.5120 - val_auc: 0.3707
Epoch 7/40
857/857 [==============================] - 97s 113ms/step - loss: 0.3940 - auc: 0.8975 - val_loss: 0.5354 - val_auc: 0.8279
Epoch 8/40
33/33 [==============================] - 1s

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_64_0.001/assets


Epoch 1/40
108/108 [==============================] - 99s 837ms/step - loss: 0.9061 - auc: 0.5773 - val_loss: 1.7053 - val_auc: 0.1709
Epoch 2/40
108/108 [==============================] - 91s 838ms/step - loss: 0.5452 - auc: 0.7878 - val_loss: 3.3564 - val_auc: 0.1689
Epoch 3/40
108/108 [==============================] - 90s 830ms/step - loss: 0.4684 - auc: 0.8516 - val_loss: 1.4462 - val_auc: 0.1779
Epoch 4/40
108/108 [==============================] - 89s 827ms/step - loss: 0.4273 - auc: 0.8775 - val_loss: 2.6673 - val_auc: 0.1688
Epoch 5/40
108/108 [==============================] - 89s 826ms/step - loss: 0.4028 - auc: 0.8915 - val_loss: 4.1572 - val_auc: 0.1661
Epoch 6/40
108/108 [==============================] - 91s 839ms/step - loss: 0.3917 - auc: 0.8984 - val_loss: 4.8161 - val_auc: 0.1732
Epoch 7/40
108/108 [==============================] - 90s 831ms/step - loss: 0.4091 - auc: 0.8883 - val_loss: 8.9467 - val_auc: 0.1692
Epoch 8/40
33/33 [==============================] - 1s 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/saved_model/smote_full_audio_coughvid_mel_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coughvid_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 6s 45ms/step - loss: 0.4800 - auc: 0.8416


[0.47997525334358215, 0.8416354060173035]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coughvid_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4682 - auc: 0.8519


[0.46824517846107483, 0.851918637752533]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coughvid_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 39ms/step - loss: 0.4573 - auc: 0.8501


[0.45729416608810425, 0.8500904440879822]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coughvid_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4565 - auc: 0.8522


[0.4564799666404724, 0.8521689772605896]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_full_audio_coughvid_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4552 - auc: 0.8385


[0.4551868736743927, 0.8384881019592285]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coughvid_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4521 - auc: 0.8495


[0.4521360695362091, 0.8495354056358337]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coughvid_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4714 - auc: 0.8370


[0.4714164435863495, 0.8369677066802979]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coughvid_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4510 - auc: 0.8549


[0.4509568214416504, 0.8548864126205444]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coughvid_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4880 - auc: 0.8366


[0.487985223531723, 0.8366490602493286]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'full_audio_coughvid_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.4517 - auc: 0.8489


[0.45173224806785583, 0.8488538265228271]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coughvid_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 9.2554 - auc: 0.1729


[9.255423545837402, 0.17287397384643555]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coughvid_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.8848 - auc: 0.6033


[0.8847925066947937, 0.603294849395752]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coughvid_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 3s 40ms/step - loss: 0.4737 - auc: 0.8467


[0.47365155816078186, 0.8467472195625305]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coughvid_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.5583 - auc: 0.8130


[0.5583159923553467, 0.8130238056182861]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_full_audio_coughvid_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

33/33 [==============================] - 2s 40ms/step - loss: 0.6113 - auc: 0.7395


[0.6113383769989014, 0.7395310401916504]

# Chunks
* potentials: use `num_chunk` to explore as long audio influence the model'accuracy (the more `num_chunk` the longer the audio)

## Feature

### Set up

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output, Audio
import IPython.display as ipd
import os
import numpy as np
import shutil
import pandas as pd
import logging
import warnings

warnings.filterwarnings("ignore")


logger = logging.getLogger('COUGHVID - HYBRID MODEL CHUNKS')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)
pd.set_option('display.max_colwidth', None)
clear_output()

In [ ]:
num_mfcc = 0

root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks'
data_root = '/content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/data preprocessing'

meta_coughvid = os.path.join(data_root, 'cough_chunking/coughvid/coughvid_chunk_metadata.csv')

coughvid_feature = [
                    os.path.join(root, 'melspectrogram_coughvid_features_20k.npy'),
                    os.path.join(root, 'melspectrogram_coughvid_features_20k_40k.npy'),
                    os.path.join(root, 'melspectrogram_coughvid_features_40k_60k.npy'),
                    os.path.join(root, 'melspectrogram_coughvid_features_60k_75k.npy'),
                    os.path.join(root, 'melspectrogram_coughvid_features_9981.npy'),
]

# mfcc_coswara = os.path.join(root, f'mfcc_{num_mfcc}_coswara_features.npy')
# mfcc_coughvid_1 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_20k.npy')
# mfcc_coughvid_2 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_20k_40k.npy')
# mfcc_coughvid_3 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_40k_60k.npy')
# mfcc_coughvid_4 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_60k_75k.npy')
# mfcc_coughvid_5 = os.path.join(root, f'mfcc_{num_mfcc}_coughvid_features_9981.npy')


### Feature preparation

#### Metadata

In [ ]:
# process metadata
meta_df = pd.read_csv(meta_coughvid)
meta_df.head()

,uuid,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,quality_1,...,choking_3,congestion_3,nothing_3,cough_type_3,diagnosis_3,severity_3,label,sr_cough,chunk_path,num_chunk
0,00039425-7f3a-42aa-ac13-834aaa2b6b92-0,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-0.wav,3
1,00039425-7f3a-42aa-ac13-834aaa2b6b92-1,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-1.wav,3
2,00039425-7f3a-42aa-ac13-834aaa2b6b92-2,2020-04-13t21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/00039425-7f3a-42aa-ac13-834aaa2b6b92.webm,coughvid_cough_chunk/00039425-7f3a-42aa-ac13-834aaa2b6b92-2.wav,3
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-0.wav,3
4,0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1,2020-04-12t04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,healthy,coughvid_cough_data/0009eb28-d8be-4dc1-92bb-907e53bc5c7a.webm,coughvid_cough_chunk/0009eb28-d8be-4dc1-92bb-907e53bc5c7a-1.wav,3


In [ ]:
logger.info(meta_df.shape)

2022-05-01 10:59:48,636 - COUGHVID - HYBRID MODEL CHUNKS - INFO - (84981, 43)


In [ ]:
DROP_COLS = [
             'uuid',
             'num_chunk',
             'datetime',
             'chunk_path',
             'sr_cough',
             'cough_detected',
             ]

meta_df.isna().sum()

uuid                         0
datetime                     0
cough_detected               0
latitude                 33197
longitude                33197
age                      35707
gender                   31986
respiratory_condition    31986
fever_muscle_pain        31986
quality_1                80743
cough_type_1             80743
dyspnea_1                80743
wheezing_1               80743
stridor_1                80743
choking_1                80743
congestion_1             80743
nothing_1                80743
diagnosis_1              80743
severity_1               80743
quality_2                80643
cough_type_2             80651
dyspnea_2                80643
wheezing_2               80643
stridor_2                80643
choking_2                80643
congestion_2             80643
nothing_2                80643
diagnosis_2              80643
severity_2               80643
quality_3                80594
dyspnea_3                80594
wheezing_3               80594
stridor_

In [ ]:
meta_df.dtypes

uuid                      object
datetime                  object
cough_detected           float64
latitude                 float64
longitude                float64
age                      float64
gender                    object
respiratory_condition     object
fever_muscle_pain         object
quality_1                 object
cough_type_1              object
dyspnea_1                 object
wheezing_1                object
stridor_1                 object
choking_1                 object
congestion_1              object
nothing_1                 object
diagnosis_1               object
severity_1                object
quality_2                 object
cough_type_2              object
dyspnea_2                 object
wheezing_2                object
stridor_2                 object
choking_2                 object
congestion_2              object
nothing_2                 object
diagnosis_2               object
severity_2                object
quality_3                 object
dyspnea_3 

In [ ]:
# fill na with unknown value
# meta_df[['latitude', 'longitude', 'age']] = meta_df[['latitude', 'longitude', 'age']].fillna(99999)
meta_df.fillna('unknown', inplace=True)
meta_df.isna().sum()

uuid                     0
datetime                 0
cough_detected           0
latitude                 0
longitude                0
age                      0
gender                   0
respiratory_condition    0
fever_muscle_pain        0
quality_1                0
cough_type_1             0
dyspnea_1                0
wheezing_1               0
stridor_1                0
choking_1                0
congestion_1             0
nothing_1                0
diagnosis_1              0
severity_1               0
quality_2                0
cough_type_2             0
dyspnea_2                0
wheezing_2               0
stridor_2                0
choking_2                0
congestion_2             0
nothing_2                0
diagnosis_2              0
severity_2               0
quality_3                0
dyspnea_3                0
wheezing_3               0
stridor_3                0
choking_3                0
congestion_3             0
nothing_3                0
cough_type_3             0
d

In [ ]:
# drop cols
meta_df.drop(DROP_COLS, axis=1, inplace=True)

In [ ]:
# take labels
meta_df.label = meta_df.label.map({
    'healthy': 0,
    'symptomatic': 1,
    'covid-19': 1
})
logger.info(meta_df.label.value_counts().to_dict())

2022-05-01 10:59:49,177 - COUGHVID - HYBRID MODEL CHUNKS - INFO - {0.0: 39866, 1.0: 13129}


In [ ]:
meta_df.label.value_counts().sum()

52995

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in meta_df.drop(['label'], axis=1).columns:
  meta_df[col] = meta_df[col].astype('string')
  meta_df[col] = le.fit_transform(meta_df[col].values)

In [ ]:
meta_feature = np.array(meta_df.drop(['label'], axis=1))

#### Features

In [ ]:
mel_feature_1 = np.load(coughvid_feature[0])
mel_feature_2 = np.load(coughvid_feature[1])
mel_feature_3 = np.load(coughvid_feature[2])
mel_feature_4 = np.load(coughvid_feature[3])
mel_feature_5 = np.load(coughvid_feature[4])

##### 2d

In [ ]:
null_label = meta_df[meta_df['label'].isna() == False].index.tolist() # index of records not n/a labels
meta_df.dropna(inplace=True)
meta_df.reset_index(drop=True, inplace=True)

In [ ]:
label_0 = meta_df[meta_df['label']==0].copy().reset_index(drop=True)
label_1 = meta_df[meta_df['label']==1].copy().reset_index(drop=True)

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_1)}, {len(index_0)}')

2022-05-01 11:01:02,512 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 13129, 39866


In [ ]:
# remove features of null labels
mel_feature_1 = mel_feature_1[[i for i in null_label if i < 20000]]
mel_feature_2 = mel_feature_2[[i - 20000 for i in null_label if i >= 20000 and i < 40000]]
mel_feature_3 = mel_feature_3[[i - 40000 for i in null_label if i >= 40000 and i < 60000]]
mel_feature_4 = mel_feature_4[[i - 60000 for i in null_label if i >= 60000 and i < 75000]]
mel_feature_5 = mel_feature_5[[i - 75000 for i in null_label if i >= 75000]]
mel_features = np.concatenate((mel_feature_1, mel_feature_2, mel_feature_3, mel_feature_4, mel_feature_5), axis=0)

In [ ]:
# release RAM
del mel_feature_1
del mel_feature_2
del mel_feature_3
del mel_feature_4
del mel_feature_5

In [ ]:
# get index of each label 0, 1 to match with extracted features
index_1 = meta_df[meta_df['label']==1].index.tolist()
index_0 = meta_df[meta_df['label']==0].index.tolist()
logger.info(f'{len(index_0)}, {len(index_1)}')

2022-05-01 11:01:18,060 - COUGHVID - HYBRID MODEL CHUNKS - INFO - 39866, 13129


In [ ]:
mel_features.shape

(52995, 224, 236)

In [ ]:
# get features for each class
feature_0 = mel_features[index_0]
feature_1 = mel_features[index_1]
del mel_features

In [ ]:
# split data and reshape feature
def data_split(feature_0, feature_1, label0, label1):
  train = np.concatenate((feature_0[:31893], feature_1[:10517]), axis=0)
  val = np.concatenate((feature_0[31893:35879], feature_1[10517:11824]), axis=0)
  test = np.concatenate((feature_0[35879:], feature_1[11824:]), axis=0)

  y_train = pd.concat([label_0[:31893], label_1[:10517]], ignore_index=True)
  y_val = pd.concat([label_0[31893:35879], label_1[10517:11824]], ignore_index=True)
  y_test = pd.concat([label_0[35879:], label_1[11824:]], ignore_index=True)

  return train, val, test,\
         np.array(y_train.label), np.array(y_val.label), np.array(y_test.label)

In [ ]:
X_train_2d, X_val_2d, X_test_2d, y_train_2d, y_val_2d, y_test_2d = data_split(feature_0, feature_1, label_0, label_1)
print(X_train_2d.shape)

(42410, 224, 236)


In [ ]:
y_train_2d

array([0., 0., 0., ..., 1., 1., 1.])

In [ ]:
del feature_0
del feature_1

##### 1d

In [ ]:
feature_0 = meta_feature[index_0]
feature_1 = meta_feature[index_1]

In [ ]:
X_train_1d, X_val_1d, X_test_1d, y_train_1d, y_val_1d, y_test_1d = data_split(feature_0, feature_1, label_0, label_1)

In [ ]:
X_train_2d = X_train_2d[20000:]
y_train_2d = y_train_2d[20000:]
X_train_2d.shape, y_train_2d.shape

((22410, 224, 236), (22410,))

In [ ]:
X_train_1d = X_train_1d[20000:]
y_train_1d = y_train_1d[20000:]
X_train_1d.shape, y_train_1d.shape

((22410, 36), (22410,))

In [ ]:
from sklearn.utils import shuffle

X_train_2d, X_train_1d, y_train_2d, y_train_1d = shuffle(X_train_2d, X_train_1d, y_train_2d, y_train_1d, random_state=42)
X_val_2d, X_val_1d, y_val_2d, y_val_1d = shuffle(X_val_2d, X_val_1d, y_val_2d, y_val_1d, random_state=42)
X_test_2d, X_test_1d, y_test_2d, y_test_1d = shuffle(X_test_2d, X_test_1d, y_test_2d, y_test_1d, random_state=42)

## Model

In [ ]:
X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(32410, 224, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(32410, 36, 1)

In [ ]:
X_train_1d.shape

(32410, 36, 1)

In [ ]:
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=5)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'coughvid_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
4052/4052 [==============================] - 249s 57ms/step - loss: 0.6492 - auc: 0.6705 - val_loss: 0.5839 - val_auc: 0.7420
Epoch 2/40
4052/4052 [==============================] - 232s 57ms/step - loss: 0.6335 - auc: 0.6756 - val_loss: 0.5772 - val_auc: 0.7448
Epoch 3/40
4052/4052 [==============================] - 232s 57ms/step - loss: 0.6329 - auc: 0.6760 - val_loss: 0.5747 - val_auc: 0.7661
Epoch 4/40
4052/4052 [==============================] - 232s 57ms/step - loss: 0.6321 - auc: 0.6786 - val_loss: 0.5747 - val_auc: 0.7662
Epoch 5/40
4052/4052 [==============================] - 233s 58ms/step - loss: 0.6323 - auc: 0.6750 - val_loss: 0.5682 - val_auc: 0.7663
Epoch 6/40
4052/4052 [==============================] - 231s 57ms/step - loss: 0.6323 - auc: 0.6731 - val_loss: 0.5748 - val_auc: 0.7441
Epoch 7/40
4052/4052 [==============================] - 231s 57ms/step - loss: 0.6319 - auc: 0.6746 - val_loss: 0.5749 - val_auc: 0.7472
Epoch 8/40
166/166 [=====================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_8_0.001/assets


Epoch 1/40
2026/2026 [==============================] - 199s 95ms/step - loss: 0.6540 - auc: 0.6931 - val_loss: 0.5741 - val_auc: 0.7552
Epoch 2/40
2026/2026 [==============================] - 191s 94ms/step - loss: 0.6325 - auc: 0.6773 - val_loss: 0.5799 - val_auc: 0.7597
Epoch 3/40
2026/2026 [==============================] - 191s 94ms/step - loss: 0.6321 - auc: 0.6768 - val_loss: 0.5714 - val_auc: 0.7443
Epoch 4/40
2026/2026 [==============================] - 191s 94ms/step - loss: 0.6320 - auc: 0.6771 - val_loss: 0.5727 - val_auc: 0.7427
Epoch 5/40
2026/2026 [==============================] - 191s 94ms/step - loss: 0.6319 - auc: 0.6753 - val_loss: 0.5712 - val_auc: 0.7430
Epoch 6/40
2026/2026 [==============================] - 189s 93ms/step - loss: 0.6319 - auc: 0.6750 - val_loss: 0.5756 - val_auc: 0.7517
Epoch 7/40
166/166 [==============================] - 5s 27ms/step - loss: 0.5810 - auc: 0.7526


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_64_0.001/assets


Epoch 1/40
254/254 [==============================] - 169s 635ms/step - loss: 0.7174 - auc: 0.6825 - val_loss: 0.6312 - val_auc: 0.7353
Epoch 2/40
254/254 [==============================] - 158s 623ms/step - loss: 0.6313 - auc: 0.6867 - val_loss: 0.6224 - val_auc: 0.7489
Epoch 3/40
254/254 [==============================] - 159s 626ms/step - loss: 0.6284 - auc: 0.6950 - val_loss: 0.5766 - val_auc: 0.7571
Epoch 4/40
254/254 [==============================] - 159s 625ms/step - loss: 0.6267 - auc: 0.6998 - val_loss: 0.5848 - val_auc: 0.7651
Epoch 5/40
254/254 [==============================] - 158s 623ms/step - loss: 0.6257 - auc: 0.7027 - val_loss: 0.5631 - val_auc: 0.7703
Epoch 6/40
254/254 [==============================] - 159s 627ms/step - loss: 0.6236 - auc: 0.7075 - val_loss: 0.5647 - val_auc: 0.7696
Epoch 7/40
254/254 [==============================] - 159s 626ms/step - loss: 0.6220 - auc: 0.7099 - val_loss: 0.5896 - val_auc: 0.7683
Epoch 8/40
254/254 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/coughvid_mel_hybrid_model_40_128_0.001/assets


## Standard scaler model

In [ ]:
X_train_2d.shape

(32410, 224, 236)

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_2d = sc.fit_transform(X_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_2d = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_2d = sc.transform(X_val_2d)
X_val_2d = np.reshape(X_val_2d, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_2d = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_2d = sc.transform(X_test_2d)
X_test_2d = np.reshape(X_test_2d, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(32410, 224, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(32410, 36, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([32410, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './coughvid_mel_hybrid_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'std_coughvid_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
               loss=loss,
               metrics=[auc])

Epoch 1/40
4052/4052 [==============================] - 250s 58ms/step - loss: 0.6455 - auc: 0.6752 - val_loss: 0.5712 - val_auc: 0.7370
Epoch 2/40
4052/4052 [==============================] - 233s 57ms/step - loss: 0.6333 - auc: 0.6762 - val_loss: 0.5700 - val_auc: 0.7585
Epoch 3/40
4052/4052 [==============================] - 230s 57ms/step - loss: 0.6328 - auc: 0.6757 - val_loss: 0.5771 - val_auc: 0.7384
Epoch 4/40
4052/4052 [==============================] - 228s 56ms/step - loss: 0.6327 - auc: 0.6746 - val_loss: 0.5765 - val_auc: 0.7434
Epoch 5/40
4052/4052 [==============================] - 229s 57ms/step - loss: 0.6325 - auc: 0.6749 - val_loss: 0.5766 - val_auc: 0.7469
Epoch 6/40
4052/4052 [==============================] - 229s 56ms/step - loss: 0.6320 - auc: 0.6762 - val_loss: 0.5725 - val_auc: 0.7662
Epoch 7/40
4052/4052 [==============================] - 229s 57ms/step - loss: 0.6317 - auc: 0.6768 - val_loss: 0.5689 - val_auc: 0.7605
Epoch 8/40
4052/4052 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_8_0.001/assets


Epoch 1/40
2026/2026 [==============================] - 198s 95ms/step - loss: 0.6593 - auc: 0.6899 - val_loss: 0.5845 - val_auc: 0.7688
Epoch 2/40
2026/2026 [==============================] - 190s 94ms/step - loss: 0.6317 - auc: 0.6810 - val_loss: 0.5753 - val_auc: 0.7655
Epoch 3/40
2026/2026 [==============================] - 190s 94ms/step - loss: 0.6319 - auc: 0.6796 - val_loss: 0.5711 - val_auc: 0.7681
Epoch 4/40
2026/2026 [==============================] - 190s 94ms/step - loss: 0.6321 - auc: 0.6768 - val_loss: 0.5747 - val_auc: 0.7678
Epoch 5/40
2026/2026 [==============================] - 190s 94ms/step - loss: 0.6320 - auc: 0.6770 - val_loss: 0.5711 - val_auc: 0.7613
Epoch 6/40
166/166 [==============================] - 4s 25ms/step - loss: 0.5672 - auc: 0.7523


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_64_0.001/assets


Epoch 1/40
254/254 [==============================] - 171s 643ms/step - loss: 0.7360 - auc: 0.6813 - val_loss: 0.5943 - val_auc: 0.7402
Epoch 2/40
254/254 [==============================] - 162s 636ms/step - loss: 0.6315 - auc: 0.6861 - val_loss: 0.5855 - val_auc: 0.7646
Epoch 3/40
254/254 [==============================] - 162s 638ms/step - loss: 0.6280 - auc: 0.6964 - val_loss: 0.5761 - val_auc: 0.7746
Epoch 4/40
254/254 [==============================] - 161s 634ms/step - loss: 0.6259 - auc: 0.7008 - val_loss: 0.5738 - val_auc: 0.7658
Epoch 5/40
254/254 [==============================] - 162s 636ms/step - loss: 0.6236 - auc: 0.7061 - val_loss: 0.5699 - val_auc: 0.7711
Epoch 6/40
254/254 [==============================] - 162s 637ms/step - loss: 0.6212 - auc: 0.7117 - val_loss: 0.5743 - val_auc: 0.7724
Epoch 7/40
254/254 [==============================] - 161s 633ms/step - loss: 0.6195 - auc: 0.7134 - val_loss: 0.5693 - val_auc: 0.7752
Epoch 8/40
166/166 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/std_coughvid_mel_hybrid_model_40_128_0.001/assets


## SMOTE

In [ ]:
X_train_2d.shape

(22410, 224, 236)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=42)

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_2d = np.reshape(X_train_2d, newshape=(num_instances, -1))
X_train_2d, y_train_2d = sm.fit_resample(X_train_2d, y_train_2d)
X_train_2d = np.reshape(X_train_2d, newshape=(X_train_2d.shape[0], first_dim, second_dim))


num_instances, first_dim = X_train_1d.shape
X_train_1d = np.reshape(X_train_1d, newshape=(num_instances, -1))
X_train_1d, y_train_1d = sm.fit_resample(X_train_1d, y_train_1d)
X_train_1d = np.reshape(X_train_1d, newshape=(X_train_1d.shape[0], first_dim))

# num_instances, first_dim, second_dim = X_val.shape
# X_val = np.reshape(X_val, newshape=(-1, second_dim))
# X_val = sc.transform(X_val)
# X_val = np.reshape(X_val, newshape=(num_instances, first_dim, second_dim))


# num_instances, first_dim, second_dim = X_test.shape
# X_test = np.reshape(X_test, newshape=(-1, second_dim))
# X_test = sc.transform(X_test)
# X_test = np.reshape(X_test, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

(23786, 224, 236, 1)

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]
X_train_1d.shape

(23786, 36, 1)

In [ ]:
# reshape target
import tensorflow as tf

y_train_2d = tf.one_hot(y_train_2d, depth=2)
y_test_2d = tf.one_hot(y_test_2d, depth=2)
y_val_2d = tf.one_hot(y_val_2d, depth=2)

In [ ]:
y_train_2d.shape

TensorShape([23786, 2])

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')
EPOCHS = 40
PATIENCE = 5
LEARNING_RATE = 0.001

In [ ]:
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Concatenate
from datetime import datetime

from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, Activation, AvgPool2D, Flatten, Dense
from tensorflow.keras.layers import Dropout, LSTM, GRU, Bidirectional

from tensorflow.keras import Model



# MobileNet block

def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def hybrid_model(input_1, input_2):

  input_1 = Input(input_1[1:])

  x = Conv2D(filters = 32, kernel_size = 1, strides = 2, padding = 'same')(input_1)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  # main part of the model

  x = mobilnet_block(x, filters = 64, strides = 1)
  x = mobilnet_block(x, filters = 128, strides = 2)
  x = mobilnet_block(x, filters = 128, strides = 1)
  x = mobilnet_block(x, filters = 256, strides = 2)
  x = mobilnet_block(x, filters = 256, strides = 1)
  x = mobilnet_block(x, filters = 512, strides = 2)

  for _ in range (5):
      x = mobilnet_block(x, filters = 512, strides = 1)
  x = mobilnet_block(x, filters = 1024, strides = 2)
  x = mobilnet_block(x, filters = 1024, strides = 1)
  x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)

  x = Flatten()(x)
  x = Dense(units=500, activation='relu')(x)
  x = Dense(units=256, activation='relu')(x)
  x = Dense(units=128, activation='relu')(x)
  x = Dense(units=64, activation='relu')(x)
  # add meta
  # input_2 = Input((7380, 128000))
  input_2 = Input(input_2[1:])

  y = Conv1D(filters=64, kernel_size=(3), strides=1, padding='same')(input_2)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4)(y)
  y = Dropout(0.5)(y)

  # LFLB2
  y = Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB3
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  # LFLB4
  y = Conv1D(filters=128, kernel_size=3, strides=1, padding='same')(y)
  y = BatchNormalization()(y)
  y = Activation('elu')(y)
  y = MaxPooling1D(pool_size=4, strides=4, padding='same')(y)
  y = Dropout(0.25)(y)

  y = LSTM(units = 128, return_sequences = True, name='lstm_layer')(y)
  y = Dropout(rate = 0.2)(y) 
  y = BatchNormalization()(y)
  y = Dropout(rate = 0.2)(y)
  y = Flatten()(y)
  y = Dense(64)(y)


  merged = Concatenate()([x, y])
  output = Dense(2, activation='softmax')(merged)

  merged_model = Model(inputs=[input_1, input_2], outputs=[output])

  return merged_model

input_2d = X_train_2d.shape
input_1d = X_train_1d.shape
model = hybrid_model(input_2d, input_1d)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
auc = tf.keras.metrics.AUC()
loss = tf.keras.losses.BinaryCrossentropy()
# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics=[auc])

In [ ]:
fp = './coughvid_hybrid_mel_bestweight.h5'

check = tf.keras.callbacks.ModelCheckpoint(fp,
                                           monitor='val_auc',
                                           save_best_only=True,
                                           mode='max')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_auc",
                                                      min_delta=0.001,
                                                      mode='max',
                                                      patience=PATIENCE)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in [8, 16, 32, 64, 128]:
  BATCH_SIZE = i

  history = model.fit([X_train_2d, X_train_1d],\
                      [y_train_2d, y_train_1d],\
                      epochs=EPOCHS,\
                      batch_size=BATCH_SIZE,\
                      validation_data=([X_val_2d, X_val_1d], [y_val_2d, y_val_1d]),\
                      callbacks=[check, earlystop_callback, tensorboard_callback],\
                      shuffle=True)
  
  model.evaluate([X_test_2d, X_test_1d], [y_test_2d, y_test_1d])

  model.save(os.path.join(SAVED_MODEL_PATH, f'smote_coughvid_mel_hybrid_model_{EPOCHS}_{BATCH_SIZE}_{LEARNING_RATE}'))

  del model

  model = hybrid_model(input_2d, input_1d)

  model.compile(optimizer=opt,
                loss=loss,
                metrics=[auc])

Epoch 1/40
2974/2974 [==============================] - 192s 58ms/step - loss: 0.7107 - auc: 0.5028 - val_loss: 0.6999 - val_auc: 0.4725
Epoch 2/40
2974/2974 [==============================] - 166s 56ms/step - loss: 0.6961 - auc: 0.4995 - val_loss: 0.6895 - val_auc: 0.5789
Epoch 3/40
2974/2974 [==============================] - 166s 56ms/step - loss: 0.6963 - auc: 0.4992 - val_loss: 0.7058 - val_auc: 0.4030
Epoch 4/40
2974/2974 [==============================] - 171s 58ms/step - loss: 0.6957 - auc: 0.4999 - val_loss: 0.6916 - val_auc: 0.5593
Epoch 5/40
2974/2974 [==============================] - 173s 58ms/step - loss: 0.6952 - auc: 0.5054 - val_loss: 0.7066 - val_auc: 0.4459
Epoch 6/40
2974/2974 [==============================] - 171s 58ms/step - loss: 0.6953 - auc: 0.4947 - val_loss: 0.6927 - val_auc: 0.5061
Epoch 7/40
2974/2974 [==============================] - 174s 58ms/step - loss: 0.6951 - auc: 0.5009 - val_loss: 0.6790 - val_auc: 0.7530
Epoch 8/40
2974/2974 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_8_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_8_0.001/assets


Epoch 1/40
1487/1487 [==============================] - 155s 99ms/step - loss: 0.7229 - auc: 0.5008 - val_loss: 0.6972 - val_auc: 0.4288
Epoch 2/40
1487/1487 [==============================] - 143s 96ms/step - loss: 0.6950 - auc: 0.5032 - val_loss: 0.7134 - val_auc: 0.4538
Epoch 3/40
1487/1487 [==============================] - 142s 96ms/step - loss: 0.6954 - auc: 0.4949 - val_loss: 0.6993 - val_auc: 0.3778
Epoch 4/40
1487/1487 [==============================] - 143s 96ms/step - loss: 0.6949 - auc: 0.4985 - val_loss: 0.6771 - val_auc: 0.6319
Epoch 5/40
1487/1487 [==============================] - 143s 96ms/step - loss: 0.6950 - auc: 0.4957 - val_loss: 0.6923 - val_auc: 0.5237
Epoch 6/40
1487/1487 [==============================] - 142s 95ms/step - loss: 0.6950 - auc: 0.4945 - val_loss: 0.6777 - val_auc: 0.7285
Epoch 7/40
1487/1487 [==============================] - 142s 96ms/step - loss: 0.6945 - auc: 0.5033 - val_loss: 0.7246 - val_auc: 0.2364
Epoch 8/40
1487/1487 [===================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_16_0.001/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_64_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_64_0.001/assets


Epoch 1/40
186/186 [==============================] - 126s 639ms/step - loss: 0.8003 - auc: 0.5140 - val_loss: 0.6993 - val_auc: 0.4243
Epoch 2/40
186/186 [==============================] - 117s 629ms/step - loss: 0.6947 - auc: 0.5042 - val_loss: 0.6909 - val_auc: 0.5856
Epoch 3/40
186/186 [==============================] - 116s 622ms/step - loss: 0.6941 - auc: 0.5059 - val_loss: 0.6763 - val_auc: 0.7309
Epoch 4/40
186/186 [==============================] - 117s 629ms/step - loss: 0.6932 - auc: 0.5168 - val_loss: 0.6858 - val_auc: 0.7368
Epoch 5/40
186/186 [==============================] - 117s 628ms/step - loss: 0.6923 - auc: 0.5256 - val_loss: 0.6690 - val_auc: 0.7269
Epoch 6/40
186/186 [==============================] - 117s 631ms/step - loss: 0.6916 - auc: 0.5344 - val_loss: 0.6868 - val_auc: 0.6205
Epoch 7/40
186/186 [==============================] - 117s 628ms/step - loss: 0.6906 - auc: 0.5431 - val_loss: 0.7160 - val_auc: 0.3920
Epoch 8/40
186/186 [============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_128_0.001/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/Senior Thesis/Workspace/recognition/cnn/mobileNet/chunks/saved_model/smote_coughvid_mel_hybrid_model_40_128_0.001/assets


## Evaluation

In [ ]:
SAVED_MODEL_PATH = os.path.join(root, 'saved_model')

### Standard scaler

In [ ]:
X_train_1d, X_val_1d, X_test_1d = X_train_1d[..., np.newaxis], X_val_1d[..., np.newaxis], X_test_1d[..., np.newaxis]

In [ ]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()

num_instances, first_dim, second_dim = X_train_2d.shape
X_train_std = np.reshape(X_train_2d, newshape=(-1, second_dim))
X_train_std = sc.fit_transform(X_train_std)
X_train_std = np.reshape(X_train_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_val_2d.shape
X_val_std = np.reshape(X_val_2d, newshape=(-1, second_dim))
X_val_std = sc.transform(X_val_std)
X_val_std = np.reshape(X_val_std, newshape=(num_instances, first_dim, second_dim))


num_instances, first_dim, second_dim = X_test_2d.shape
X_test_std = np.reshape(X_test_2d, newshape=(-1, second_dim))
X_test_std = sc.transform(X_test_std)
X_test_std = np.reshape(X_test_std, newshape=(num_instances, first_dim, second_dim))

# reshape training data

X_train_std, X_val_std, X_test_std = X_train_std[..., np.newaxis], X_val_std[..., np.newaxis], X_test_std[..., np.newaxis]

# reshape target
import tensorflow as tf

y_train_std = tf.one_hot(y_train_2d, depth=2)
y_test_std = tf.one_hot(y_test_2d, depth=2)
y_val_std = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coughvid_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 14s 28ms/step - loss: 0.5857 - auc: 0.7643


[0.5857377052307129, 0.7643204927444458]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coughvid_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 27ms/step - loss: 0.5739 - auc: 0.7697


[0.5738838911056519, 0.7696845531463623]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coughvid_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5782 - auc: 0.7589


[0.5782453417778015, 0.7589149475097656]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coughvid_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5672 - auc: 0.7522


[0.5672239065170288, 0.7522109150886536]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'std_coughvid_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_std, X_test_1d], [y_test_std, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5713 - auc: 0.7563


[0.5712579488754272, 0.7562891244888306]

### Raw features

In [ ]:
# reshape training data

X_train_2d, X_val_2d, X_test_2d = X_train_2d[..., np.newaxis], X_val_2d[..., np.newaxis], X_test_2d[..., np.newaxis]
X_train_2d.shape

# reshape target
import tensorflow as tf

y_train = tf.one_hot(y_train_2d, depth=2)
y_test = tf.one_hot(y_test_2d, depth=2)
y_val = tf.one_hot(y_val_2d, depth=2)

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coughvid_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5785 - auc: 0.7683


[0.5784988403320312, 0.7682523727416992]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coughvid_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5780 - auc: 0.7657


[0.5780225992202759, 0.7656878232955933]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coughvid_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5699 - auc: 0.7707


[0.5699368119239807, 0.7706869840621948]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coughvid_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5810 - auc: 0.7526


[0.5809841752052307, 0.7526347041130066]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'coughvid_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.5643 - auc: 0.7571


[0.5642850995063782, 0.7570953369140625]

### SMOTE

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coughvid_mel_hybrid_model_40_128_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6950 - auc: 0.5032


[0.6949606537818909, 0.5032264590263367]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coughvid_mel_hybrid_model_40_64_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6941 - auc: 0.6557


[0.6941270232200623, 0.6557106971740723]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coughvid_mel_hybrid_model_40_32_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6987 - auc: 0.4381


[0.6986925005912781, 0.4380578100681305]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coughvid_mel_hybrid_model_40_16_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6769 - auc: 0.6759


[0.6768510341644287, 0.6759255528450012]

In [ ]:
loaded_model = tf.keras.models.load_model(os.path.join(SAVED_MODEL_PATH,
                                                       'smote_coughvid_mel_hybrid_model_40_8_0.001'))
loaded_model.evaluate([X_test_2d, X_test_1d], [y_test, y_test_1d], verbose=1)

166/166 [==============================] - 6s 28ms/step - loss: 0.6947 - auc: 0.5194


[0.6947112083435059, 0.5194021463394165]